In [1]:
import pandas as pd
import pickle

from collections import defaultdict
import string

import torch
from torch import nn
from torch.nn.functional import softmax
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer
from transformers import RobertaConfig
from datasets import Dataset
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

from IPython.display import display, HTML

from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

2025-06-29 19:51:15.427357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751226675.642558      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751226675.703195      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from huggingface_hub import login

login(new_session=False, # Won’t request token if one is already saved on machine
write_permission=True, # Requires a token with write permission
token="", # The name of your token
add_to_git_credential=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)
Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [3]:
checkpoint = "ConditionalNLI/Exp5_roberta-large-mnli"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3, device_map="auto", output_attentions=True)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/945 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [5]:
config = RobertaConfig()
batch_size = 4
criterion = nn.CrossEntropyLoss()
config.max_position_embeddings

512

## Running on Dataset

In [6]:
from datasets import load_dataset, concatenate_datasets
import json
import ast

In [7]:
test_df4 = pd.read_excel('/kaggle/input/confer-extension/Part5_Dataset/Part5_Type4.xlsx')
test_df4

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related
...,...,...,...,...,...,...,...
292,"If Christina is a makeup artist, she'll grab h...",Christina has a makeup sponge.,E,possessive,her makeup sponge,rides a scooter,unrelated
293,"If Rahim is a clergyman, he'll bring his holy ...",Rahim has a holy book.,E,possessive,his holy book,gardens roses,unrelated
294,"If James is a police officer, he'll pick his g...",James has a gun.,E,possessive,his gun,drinks tea,unrelated
295,"If Maya is a piano teacher, she'll pack her me...",Maya has a metronome.,E,possessive,her metronome,knits hats,unrelated


In [8]:
test_df5p = pd.read_excel('/kaggle/input/confer-extension/Part5_Dataset/Part5_Type5P.xlsx')
test_df5p

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,the university,related
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,post exam celebratory dinner,related
2,"If Scarlett finishes her chores early, her hus...",Scarlett has a husband.,E,possessive,her husband,lunch,related
3,"If Chloe finishes his project early, his colle...",Chloe has colleagues.,E,possessive,his colleagues,drinks,related
4,"If Charlotte passes her math test, her parents...",Charlotte has parents.,E,possessive,her parents,a calculator,related
...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated


In [9]:
test_df5a = pd.read_excel('/kaggle/input/confer-extension/Part5_Dataset/Part5_Type5A.xlsx')
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,attends a movie festival,watch a movie by Nolan again,related
1,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,finishes a mystery novel,read a book by Dan Brown again,related
2,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,bakes a Swedish pastry,eat a traditional Swedish dish again,related
3,"If Chloe attends a pop concert, she'll never s...",Chloe has sung a pop song before.,E,again,attends a pop concert,sing a pop song again,related
4,"If Charlotte attends a pop music festival, she...",Charlotte has listened to a pop song before.,E,again,attends a pop music festival,listen to a pop song again,related
...,...,...,...,...,...,...,...
795,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,again,breaks down,obtain a job position again,unrelated
796,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,again,finishes her work early,convey a confidential message again,unrelated
797,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,finishes work early,manage to keep a secret again,unrelated
798,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,misses the bus,cherish a moment in peace again,unrelated


In [10]:
test_df = pd.concat([test_df4,test_df5a,test_df5p])
test_df

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related
...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated


In [11]:
test_df.to_csv('part5_all.csv')

In [28]:
sample_test_df = test_df4.iloc[:8].reset_index()
sample_test_df 

,index,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
1,1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related
2,2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related
3,3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related
4,4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related
5,5,"If Donald is a crime scene investigator, he'll...",Donald has evidence bags.,E,possessive,his evidence bags,secure crime scene samples,related
6,6,"If Edward is a playwright, he'll use his pen t...",Edward has a pen.,E,possessive,his pen,write scripts for the stage,related
7,7,"If William is a notary, he'll use his notary s...",William has a notary seal.,E,possessive,his notary seal,certify documents,related


In [57]:
sample_test_df = test_df4[test_df4['premise'].str.contains('If Liam is an aerospace')]
sample_test_df

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
99,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,cool off on a hot summer day,somewhat_related
198,"If Liam is an aerospace engineer, he'll grab h...",Liam has a wind tunnel.,E,possessive,his wind tunnel,feeds the cat,unrelated


In [14]:
test_df = sample_test_df

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3, device_map="auto", output_attentions=True)

In [16]:
test_ds = Dataset.from_pandas(test_df)
test_ds

Dataset({
    features: ['premise', 'hypothesis', 'gold_label', 'trigger', 'key_phrase1', 'key_phrase2', 'type', '__index_level_0__'],
    num_rows: 3
})

In [17]:
columns_to_keep = ['index', 'premise', 'hypothesis']
test_ds = test_ds.remove_columns([col for col in test_ds.column_names if col not in columns_to_keep])
test_ds

Dataset({
    features: ['premise', 'hypothesis'],
    num_rows: 3
})

In [18]:
def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True, max_length=256, padding='max_length')

In [19]:
def predict_with_probs(test_df):

    test_ds = Dataset.from_pandas(test_df)
    columns_to_keep = ['index', 'premise', 'hypothesis']
    test_ds = test_ds.remove_columns([col for col in test_ds.column_names if col not in columns_to_keep])
    tokenized_test_ds = test_ds.map(tokenize_function, batched=True)
    tokenized_test_ds.set_format("torch")
    tokenized_test_ds = tokenized_test_ds.remove_columns(["premise", "hypothesis"])
    
    # Predicting Labels on Dataset
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length", max_length=256, return_tensors="pt")
    dataloader = DataLoader(tokenized_test_ds, batch_size=32, collate_fn=data_collator)
    
    model.eval()
    predictions = []
    probabilities = []
    
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to("cuda") for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            probs = softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)
            
            predictions.extend(preds.cpu().tolist())
            probabilities.extend(probs.cpu().tolist())
    
    test_df['predicted_label'] = predictions
    test_df['prediction_probs'] = probabilities
    return test_df
    

In [20]:
test_df4.to_csv('part5_type4_deberta_results_with_probs.csv')
test_df5p.to_csv('part5_type5p_deberta_results_with_probs.csv')
test_df5a.to_csv('part5_type5a_deberta_results_with_probs.csv')

In [21]:
import string

def is_content_token(token):
    # Remove special tokens 
    return token not in tokenizer.all_special_tokens

In [22]:
def compute_attention(row):

    premise = row['premise']
    hypothesis = row['hypothesis']
    inputs = tokenizer(premise, hypothesis, return_tensors='pt')

    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()} 
    
    with torch.no_grad():
        outputs = model(**inputs)
        attentions = outputs.attentions

    return inputs, attentions

In [23]:
def cls_token_attention(row):

    attention = row['attention']
    
    # Aggregating Attention scores 
    attn_tensor = torch.stack(attention)  # shape: (num_layers, batch, num_heads, seq_len, seq_len)
    cls_index = 0  # RoBERTa’s <s> token is at index 0
    cls_attention = attn_tensor[:, :, :, cls_index, :]  # shape: (num_layers, batch, num_heads, seq_len)

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    attention_values = cls_attention.mean(dim=(0,1,2)).squeeze().tolist()
    
    # Filter
    filtered_tokens = []
    filtered_attention = []
    
    for token, attn in zip(tokens, attention_values):
        if is_content_token(token):
            filtered_tokens.append(token)
            filtered_attention.append(attn)

    plt.figure(figsize=(12, 6))
    plt.bar(range(len(filtered_tokens)), filtered_attention)
    plt.xticks(range(len(filtered_tokens)), filtered_tokens, rotation=90)
    plt.title("Attention from <s> Token (Filtered to Words Only)")
    plt.tight_layout()
    plt.show()

    return attention_values

## Average Attention

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

In [81]:
def average_attention(row): 

    inputs, attention = compute_attention(row)
    
    attn_tensor = torch.stack(attention)  # Shape: (num_layers, batch, num_heads, seq_len, seq_len)
    avg_attn = attn_tensor.mean(dim=(0, 1, 2))  # Shape: (seq_len, seq_len)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    avg_attn = avg_attn.detach().cpu().numpy()    

    return pd.Series([avg_attn, tokens])

In [41]:
def is_partial_word(tok,prev_tok):
    return not tok.startswith('Ġ') \
                and tok not in string.punctuation \
                and tok not in tokenizer.all_special_tokens \
                and prev_tok not in tokenizer.all_special_tokens

In [33]:
def compute_attention_metrics(row):

    avg_attention = row['attention']
    tokens =  row['tokens']
    attention_df = pd.DataFrame(avg_attention,index=tokens,columns=tokens)

    # clean attention matrix 
    partial_words = [tok for i,tok in enumerate(attention_df.index) if not tok.startswith('Ġ') 
                and tok not in string.punctuation
                and tok not in tokenizer.all_special_tokens
                and attention_df.index[i-1] not in tokenizer.all_special_tokens]

    # if it's a partial word, add its row to the previous row (merging tokens)
    for i in range(len(attention_df)-1,0,-1):
        current_token = attention_df.index[i]
        prev_token = attention_df.index[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[i-1] += attention_df.iloc[i] # first row gets total value
            attention_df.iloc[i] = 0 # set second row to 0
            merged_token = attention_df.index.values[i-1] + attention_df.index.values[i]
            attention_df.index.values[i-1] = merged_token

    attention_df.index = [token[1:] if token.startswith('Ġ') else token for token in attention_df.index]

    # repeat for columns
    for i in range(attention_df.shape[1]-1,0,-1):
        current_token = attention_df.columns[i]
        prev_token = attention_df.columns[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[:, i-1] += attention_df.iloc[:, i]
            attention_df.iloc[:, i] = 0
            merged_token = attention_df.columns.values[i - 1] + current_token
            attention_df.columns.values[i-1] = merged_token

    attention_df.columns = [token[1:] if token.startswith('Ġ') else token for token in attention_df.columns ]

    # dropping rows and columns that aren't words
    attention_df = attention_df[attention_df.index.to_series().apply(is_content_token)]
    drop_cols = [col for col in attention_df.columns if not is_content_token(col)]
    attention_df = attention_df.drop(columns=drop_cols)

    # compute overall average attention
    avg_attention = attention_df.mean().mean()

    # compute average attention from key_phrase1 to key_phrase2 
    key_phrase1 = row['key_phrase1']
    key_phrase2 = row['key_phrase2']
    
    key_phrase1_words = key_phrase1.split()
    key_phrase2_words = key_phrase2.split()

    start_key_phrase1 = key_phrase1_words[0]
    end_key_phrase1 = key_phrase1_words[-1]
    start_key_phrase2 = key_phrase2_words[0]
    end_key_phrase2 = key_phrase2_words[-1]
    phrase_thresholds = [start_key_phrase1, end_key_phrase1, start_key_phrase2, end_key_phrase2]
    indices = {}
    try:
        indices['start_key_phrase1'] = list(attention_df.index).index(start_key_phrase1)
    except: 
        if 'his' in attention_df.index: 
            start_key_phrase1 = 'his'
        elif 'her' in attention_df.index: 
            start_key_phrase1 = 'her'
        elif 'their' in attention_df.index: 
            start_key_phrase1 = 'their'
        else:
            return pd.Series([avg_attention,None])

    try:
        indices['start_key_phrase2'] = list(attention_df.index).index(start_key_phrase2)
    except:
        if 'to' in attention_df.index: 
            indices['start_key_phrase2'] = list(attention_df.index).index('to') + 1
        else:
            return pd.Series([avg_attention,None])

    # searching for end index starts after start index
    try:
        indices['end_key_phrase1'] = indices['start_key_phrase1'] + list(attention_df.index)[indices['start_key_phrase1']:].index(end_key_phrase1) + 1
    except:
        if 'to' in attention_df.index: 
            indices['end_key_phrase1'] = list(attention_df.index).index('to')
        else:
            return pd.Series([avg_attention,None])

    try:
        indices['end_key_phrase2'] = indices['start_key_phrase2'] + list(attention_df.index)[indices['start_key_phrase2']:].index(end_key_phrase2) + 1
    except:
        if '.' in attention_df.index: 
            indices['end_key_phrase2'] = list(attention_df.index).index('.')
        else:
            return pd.Series([avg_attention,None])
        
    key_phrase_attention = attention_df.iloc[indices['start_key_phrase1']:indices['end_key_phrase1'],indices['start_key_phrase1']:indices['end_key_phrase1']]
    avg_key_phrase_attention = key_phrase_attention.mean().mean()
    
    return pd.Series([avg_attention,avg_key_phrase_attention])

In [48]:
sample_test_df[['attention','tokens']] = sample_test_df.apply(average_attention,axis=1)

/tmp/ipykernel_35/3046522313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['attention','tokens']] = sample_test_df.apply(average_attention,axis=1)


In [49]:
sample_test_df[['avg_attention','avg_key_phrase_attention']] = sample_test_df.apply(compute_attention_metrics, axis=1)
sample_test_df

/tmp/ipykernel_35/705830274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['avg_attention','avg_key_phrase_attention']] = sample_test_df.apply(compute_attention_metrics, axis=1)
/tmp/ipykernel_35/705830274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['avg_attention','avg_key_phrase_attention']] = sample_test_df.apply(compute_attention_metrics, axis=1)


,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,avg_attention,avg_key_phrase_attention
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[0.17215282, 0.0038426423, 0.005423983, 0.005...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine...",0.029764,0.041329
99,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,cool off on a hot summer day,somewhat_related,"[[0.16579756, 0.0035814724, 0.005217339, 0.004...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine...",0.026036,0.040489
198,"If Liam is an aerospace engineer, he'll grab h...",Liam has a wind tunnel.,E,possessive,his wind tunnel,feeds the cat,unrelated,"[[0.17278796, 0.003714662, 0.005465446, 0.0053...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine...",0.029752,0.042756


In [50]:
len(test_df4)

297

In [51]:
test_df4[['attention','tokens']] = test_df4.apply(average_attention,axis=1)
test_df4[['avg_attention','avg_key_phrase_attention']] = test_df4.apply(compute_attention_metrics, axis=1)
test_df4 

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,avg_attention,avg_key_phrase_attention
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[0.17215282, 0.0038426423, 0.005423983, 0.005...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine...",0.029764,0.041329
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related,"[[0.16770327, 0.0035497255, 0.0051103025, 0.00...","[<s>, If, ĠBill, Ġis, Ġa, Ġcontent, Ġstrategis...",0.026857,0.035839
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related,"[[0.17198363, 0.0034583937, 0.0059485077, 0.00...","[<s>, If, ĠSteve, Ġis, Ġa, Ġconservation, Ġoff...",0.028585,0.038049
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related,"[[0.16816372, 0.003677845, 0.0056913956, 0.004...","[<s>, If, ĠMatt, Ġis, Ġa, Ġsc, uba, Ġdiver, ,,...",0.026845,0.130412
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related,"[[0.17020833, 0.0036016905, 0.006539823, 0.004...","[<s>, If, ĠNoah, Ġis, Ġa, Ġpastry, Ġchef, ,, Ġ...",0.026637,0.038994
...,...,...,...,...,...,...,...,...,...,...,...
292,"If Christina is a makeup artist, she'll grab h...",Christina has a makeup sponge.,E,possessive,her makeup sponge,rides a scooter,unrelated,"[[0.17304003, 0.0033935383, 0.0059805214, 0.00...","[<s>, If, ĠChristina, Ġis, Ġa, Ġmakeup, Ġartis...",0.028522,0.037820
293,"If Rahim is a clergyman, he'll bring his holy ...",Rahim has a holy book.,E,possessive,his holy book,gardens roses,unrelated,"[[0.17369306, 0.0037026766, 0.008242931, 0.0, ...","[<s>, If, ĠRah, im, Ġis, Ġa, Ġclergy, man, ,, ...",0.029731,0.044614
294,"If James is a police officer, he'll pick his g...",James has a gun.,E,possessive,his gun,drinks tea,unrelated,"[[0.17721617, 0.0036237594, 0.0058197654, 0.00...","[<s>, If, ĠJames, Ġis, Ġa, Ġpolice, Ġofficer, ...",0.035098,0.051738
295,"If Maya is a piano teacher, she'll pack her me...",Maya has a metronome.,E,possessive,her metronome,knits hats,unrelated,"[[0.1704165, 0.003690166, 0.0066174604, 0.0052...","[<s>, If, ĠMaya, Ġis, Ġa, Ġpiano, Ġteacher, ,,...",0.027692,0.135981


In [52]:
test_df4['avg_attention'].mean()

0.029541366304010653

In [53]:
test_df4['avg_key_phrase_attention'].mean()

0.05236447829677648

In [54]:
test_df4.groupby('type')['avg_key_phrase_attention'].mean()

type
related             0.052250
somewhat_related    0.051439
unrelated           0.053403
Name: avg_key_phrase_attention, dtype: float64

In [55]:
test_df4.to_csv('roberta_part5_type4.csv')

## Type 5P

In [56]:
test_df5p[['attention','tokens']] = test_df5p.apply(average_attention,axis=1)
test_df5p[['avg_attention','avg_key_phrase_attention']] = test_df5p.apply(compute_attention_metrics, axis=1)
test_df5p

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,avg_attention,avg_key_phrase_attention
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,the university,related,"[[0.17674983, 0.004634804, 0.006193371, 0.0059...","[<s>, If, ĠJessica, Ġattends, Ġthe, Ġconferenc...",0.036828,0.056826
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,post exam celebratory dinner,related,"[[0.17173706, 0.003674858, 0.005905538, 0.0045...","[<s>, If, ĠEmily, Ġpasses, Ġher, Ġdriving, Ġte...",0.026978,0.024960
2,"If Scarlett finishes her chores early, her hus...",Scarlett has a husband.,E,possessive,her husband,lunch,related,"[[0.17417377, 0.004215937, 0.005900876, 0.0063...","[<s>, If, ĠScarlett, Ġfinishes, Ġher, Ġchores,...",0.033768,0.027547
3,"If Chloe finishes his project early, his colle...",Chloe has colleagues.,E,possessive,his colleagues,drinks,related,"[[0.17712025, 0.004594033, 0.008442057, 0.0061...","[<s>, If, ĠChloe, Ġfinishes, Ġhis, Ġproject, Ġ...",0.035150,0.028304
4,"If Charlotte passes her math test, her parents...",Charlotte has parents.,E,possessive,her parents,a calculator,related,"[[0.178489, 0.0046310555, 0.0077134296, 0.0060...","[<s>, If, ĠCharlotte, Ġpasses, Ġher, Ġmath, Ġt...",0.036657,0.027533
...,...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated,"[[0.1751115, 0.004851301, 0.006780968, 0.00739...","[<s>, If, ĠDorothy, Ġbike, Ġbreaks, Ġdown, ,, ...",0.033661,0.056756
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated,"[[0.17640345, 0.0040289722, 0.0055452753, 0.00...","[<s>, If, ĠSusan, Ġfinishes, Ġher, Ġassignment...",0.033486,0.040664
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated,"[[0.17341411, 0.004660236, 0.006708904, 0.0068...","[<s>, If, ĠMargaret, Ġfinishes, Ġwork, Ġearly,...",0.035444,NaN
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated,"[[0.17396751, 0.0047013992, 0.0071969805, 0.00...","[<s>, If, ĠBetty, Ġmisses, Ġthe, Ġtrain, ,, Ġh...",0.033881,0.058200


In [57]:
test_df5p['avg_key_phrase_attention'].isna().sum()

7

In [58]:
test_df5p['avg_attention'].mean()

0.033237901940010486

In [59]:
test_df5p['avg_key_phrase_attention'].mean()

0.04719024258671613

In [60]:
test_df5p.groupby('type')['avg_key_phrase_attention'].mean()

type
related      0.046818
unrelated    0.047563
Name: avg_key_phrase_attention, dtype: float64

In [61]:
test_df5p.to_csv('roberta_part5_type5p.csv')

## Type 5A

In [ ]:
test_df5a

In [ ]:
test_df5a[test_df5a['key_phrase2'].isna()]

In [62]:
test_df5a[['attention','tokens']] = test_df5a.apply(average_attention,axis=1)
test_df5a[['avg_attention','avg_key_phrase_attention']] = test_df5a.apply(compute_attention_metrics, axis=1)
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,avg_attention,avg_key_phrase_attention
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,attends a movie festival,watch a movie by Nolan again,related,"[[0.17383206, 0.0037895227, 0.0049331454, 0.00...","[<s>, If, ĠJessica, Ġattends, Ġa, Ġmovie, Ġfes...",0.028679,0.036887
1,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,finishes a mystery novel,read a book by Dan Brown again,related,"[[0.16946673, 0.0037922934, 0.0051234383, 0.00...","[<s>, If, ĠEmily, Ġfinishes, Ġa, Ġmystery, Ġno...",0.026935,0.035737
2,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,bakes a Swedish pastry,eat a traditional Swedish dish again,related,"[[0.16743962, 0.0044264817, 0.005428409, 0.009...","[<s>, If, ĠScarlett, Ġb, akes, Ġa, ĠSwedish, Ġ...",0.026787,0.034127
3,"If Chloe attends a pop concert, she'll never s...",Chloe has sung a pop song before.,E,again,attends a pop concert,sing a pop song again,related,"[[0.17697136, 0.003977977, 0.0071977293, 0.005...","[<s>, If, ĠChloe, Ġattends, Ġa, Ġpop, Ġconcert...",0.029263,0.041221
4,"If Charlotte attends a pop music festival, she...",Charlotte has listened to a pop song before.,E,again,attends a pop music festival,listen to a pop song again,related,"[[0.16895095, 0.0037841196, 0.006191456, 0.005...","[<s>, If, ĠCharlotte, Ġattends, Ġa, Ġpop, Ġmus...",0.026857,0.032774
...,...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,again,breaks down,obtain a job position again,unrelated,"[[0.17003243, 0.0043898, 0.012263201, 0.0, 0.0...","[<s>, If, ĠDorothy, 's, Ġcar, Ġbreaks, Ġdown, ...",0.028915,0.045879
796,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,again,finishes her work early,convey a confidential message again,unrelated,"[[0.17349827, 0.0040467894, 0.0056869434, 0.00...","[<s>, If, ĠSusan, Ġfinishes, Ġher, Ġwork, Ġear...",0.030974,0.041467
797,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,finishes work early,manage to keep a secret again,unrelated,"[[0.16780135, 0.0039324234, 0.006933586, 0.005...","[<s>, If, ĠMargaret, Ġfinishes, Ġwork, Ġearly,...",0.029111,0.049131
798,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,misses the bus,cherish a moment in peace again,unrelated,"[[0.16978866, 0.0040895576, 0.0071632615, 0.00...","[<s>, If, ĠBetty, Ġmisses, Ġthe, Ġbus, ,, Ġshe...",0.028906,0.050199


In [63]:
test_df5a['avg_key_phrase_attention'].isna().sum()

2

In [64]:
test_df5a['avg_attention'].mean()

0.027197212174069136

In [65]:
test_df5a['avg_key_phrase_attention'].mean()

0.039488563047988076

In [66]:
test_df5a.groupby('type')['avg_attention'].mean()

type
related      0.026200
unrelated    0.028194
Name: avg_attention, dtype: float64

In [ ]:
test_df5a.groupby('type')['avg_key_phrase_attention'].mean()

In [67]:
test_df5a.to_csv('roberta_part5_type5a.csv')

In [ ]:
test_df5a.loc[test_df5a['type'] == 'related']['avg_key_phrase_attention'].plot.hist(bins=20, edgecolor='black')

In [ ]:
test_df5a.loc[test_df5a['type'] == 'unrelated']['avg_key_phrase_attention'].plot.hist(bins=20, edgecolor='black')

In [ ]:
# split by correctly and incorrectly predicted to observe

## Attention from CLS token

In [40]:
def cls_token_attention(row):

    avg_attention = row['attention']
    tokens = row['tokens']
    attention_df = pd.DataFrame(avg_attention,index=tokens,columns=tokens)

    partial_words = [tok for i,tok in enumerate(attention_df.index) if not tok.startswith('Ġ') 
                and tok not in string.punctuation \
                and tok not in tokenizer.all_special_tokens \
                and attention_df.index[i-1] not in tokenizer.all_special_tokens]

    # merging columns
    for i in range(attention_df.shape[1]-1,0,-1):
        current_token = attention_df.columns[i]
        prev_token = attention_df.columns[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[:, i-1] += attention_df.iloc[:, i]
            attention_df.iloc[:, i] = 0
            merged_token = attention_df.columns.values[i - 1] + current_token
            attention_df.columns.values[i-1] = merged_token

    drop_cols = [col for col in attention_df.columns if col in partial_words]
    attention_df = attention_df.drop(columns=drop_cols)
    attention_df.columns = [token[1:] if token.startswith('Ġ') else token for token in attention_df.columns ]
    cls_attention = attention_df.iloc[0]

    cls_attention_dict = {}
    for key, value in cls_attention.items():
        cls_attention_dict[key] = cls_attention_dict.get(key, 0) + value

    return cls_attention_dict

In [73]:
def get_key_phrase_indices(row,attention_df):

    key_phrase1 = row['key_phrase1']
    key_phrase2 = row['key_phrase2']
    
    key_phrase1_words = key_phrase1.split()
    key_phrase2_words = key_phrase2.split()

    start_key_phrase1 = key_phrase1_words[0]
    end_key_phrase1 = key_phrase1_words[-1]
    start_key_phrase2 = key_phrase2_words[0]
    end_key_phrase2 = key_phrase2_words[-1]
    phrase_thresholds = [start_key_phrase1, end_key_phrase1, start_key_phrase2, end_key_phrase2]
    indices = {}
    try:
        indices['start_key_phrase1'] = list(attention_df.index).index(start_key_phrase1)
    except: 
        if 'his' in attention_df.index: 
            start_key_phrase1 = 'his'
        elif 'her' in attention_df.index: 
            start_key_phrase1 = 'her'
        elif 'their' in attention_df.index: 
            start_key_phrase1 = 'their'
        else:
            indices['start_key_phrase1'] = None
        indices['start_key_phrase1'] = list(attention_df.index).index(start_key_phrase1)

    try:
        indices['start_key_phrase2'] = list(attention_df.index).index(start_key_phrase2)
    except:
        if 'to' in attention_df.index: 
            indices['start_key_phrase2'] = list(attention_df.index).index('to') + 1
        else:
            indices['start_key_phrase2'] = None

    # searching for end index starts after start index
    try:
        indices['end_key_phrase1'] = indices['start_key_phrase1'] + list(attention_df.index)[indices['start_key_phrase1']:].index(end_key_phrase1) + 1
    except:
        if 'to' in attention_df.index: 
            indices['end_key_phrase1'] = list(attention_df.index).index('to')
        else:
            indices['end_key_phrase1'] = None 

    try:
        indices['end_key_phrase2'] = indices['start_key_phrase2'] + list(attention_df.index)[indices['start_key_phrase2']:].index(end_key_phrase2) + 1
    except:
        if '.' in attention_df.index: 
            indices['end_key_phrase2'] = list(attention_df.index).index('.')
        else:
            indices['end_key_phrase2'] = None 

    return indices

    

In [78]:
def roberta_key_attention_col(row):

    avg_attention = row['attention']
    tokens = row['tokens']
    attention_df = pd.DataFrame(avg_attention,index=tokens,columns=tokens)

    partial_words = [tok for i,tok in enumerate(attention_df.index) if not tok.startswith('Ġ') 
                and tok not in string.punctuation \
                and tok not in tokenizer.all_special_tokens \
                and attention_df.index[i-1] not in tokenizer.all_special_tokens]

    # if it's a partial word, add its row to the previous row (merging tokens)
    for i in range(len(attention_df)-1,0,-1):
        current_token = attention_df.index[i]
        prev_token = attention_df.index[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[i-1] += attention_df.iloc[i] # first row gets total value
            attention_df.iloc[i] = 0 # set second row to 0
            merged_token = attention_df.index.values[i-1] + attention_df.index.values[i]
            attention_df.index.values[i-1] = merged_token

    attention_df.index = [token[1:] if token.startswith('Ġ') else token for token in attention_df.index]
    # attention_df = attention_df[attention_df.index.to_series().apply(lambda x: x not in partial_words)]

    # merging columns
    for i in range(attention_df.shape[1]-1,0,-1):
        current_token = attention_df.columns[i]
        prev_token = attention_df.columns[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[:, i-1] += attention_df.iloc[:, i]
            attention_df.iloc[:, i] = 0
            merged_token = attention_df.columns.values[i - 1] + current_token
            attention_df.columns.values[i-1] = merged_token

    # drop_cols = [col for col in attention_df.columns if col in partial_words]
    # attention_df = attention_df.drop(columns=drop_cols)
    attention_df.columns = [token[1:] if token.startswith('Ġ') else token for token in attention_df.columns ]
    cls_attention = attention_df[['<s>','.']] # three important columns: the first and the '.' columns 
    cls_attention = cls_attention.mean(axis=1)
    avg_class_attention = cls_attention.mean()

    # compute average attention from key_phrase1 and key_phrase2 
    indices = get_key_phrase_indices(row,attention_df)

    if indices['start_key_phrase1'] and indices['end_key_phrase1']:
        key_phrase1_att = cls_attention.iloc[indices['start_key_phrase1']:indices['end_key_phrase1']]
        key_phrase1_att = key_phrase1_att.mean()
    else:
        key_phrase1_att = None 

    if indices['start_key_phrase2'] and indices['end_key_phrase2']:
        key_phrase2_att = cls_attention.iloc[indices['start_key_phrase2']:indices['end_key_phrase2']]
        key_phrase2_att = key_phrase2_att.mean()
    else:
        key_phrase2_att = None

    return pd.Series([cls_attention,avg_class_attention,key_phrase1_att,key_phrase2_att])

In [31]:
sample_test_df

,index,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens
0,0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[0.17215282, 0.0038426423, 0.005423983, 0.005...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine..."
1,1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related,"[[0.16770327, 0.0035497255, 0.0051103025, 0.00...","[<s>, If, ĠBill, Ġis, Ġa, Ġcontent, Ġstrategis..."
2,2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related,"[[0.17198363, 0.0034583937, 0.0059485077, 0.00...","[<s>, If, ĠSteve, Ġis, Ġa, Ġconservation, Ġoff..."
3,3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related,"[[0.16816372, 0.003677845, 0.0056913956, 0.004...","[<s>, If, ĠMatt, Ġis, Ġa, Ġsc, uba, Ġdiver, ,,..."
4,4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related,"[[0.17020833, 0.0036016905, 0.006539823, 0.004...","[<s>, If, ĠNoah, Ġis, Ġa, Ġpastry, Ġchef, ,, Ġ..."
5,5,"If Donald is a crime scene investigator, he'll...",Donald has evidence bags.,E,possessive,his evidence bags,secure crime scene samples,related,"[[0.17583454, 0.003670351, 0.005651432, 0.0048...","[<s>, If, ĠDonald, Ġis, Ġa, Ġcrime, Ġscene, Ġi..."
6,6,"If Edward is a playwright, he'll use his pen t...",Edward has a pen.,E,possessive,his pen,write scripts for the stage,related,"[[0.17331609, 0.00359642, 0.00496802, 0.004719...","[<s>, If, ĠEdward, Ġis, Ġa, Ġplay, wright, ,, ..."
7,7,"If William is a notary, he'll use his notary s...",William has a notary seal.,E,possessive,his notary seal,certify documents,related,"[[0.1739344, 0.004389406, 0.0056128902, 0.0050...","[<s>, If, ĠWilliam, Ġis, Ġa, Ġnot, ary, ,, Ġhe..."


In [79]:
sample_test_df[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = sample_test_df.apply(roberta_key_attention_col,axis=1)
sample_test_df

/tmp/ipykernel_35/272735351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = sample_test_df.apply(roberta_key_attention_col,axis=1)
/tmp/ipykernel_35/272735351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = sample_test_df.apply(roberta_key_attention_col,axis=1)


,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention,key_phrase1_att,key_phrase2_att,avg_cls_attention
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[0.17215282, 0.0038426423, 0.005423983, 0.005...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine...",<s> 0.213910 If 0.17259...,0.172267,0.177741,0.176148
99,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,cool off on a hot summer day,somewhat_related,"[[0.16579756, 0.0035814724, 0.005217339, 0.004...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine...",<s> 0.211831 If 0.171387 Li...,0.171611,0.166738,0.172743
198,"If Liam is an aerospace engineer, he'll grab h...",Liam has a wind tunnel.,E,possessive,his wind tunnel,feeds the cat,unrelated,"[[0.17278796, 0.003714662, 0.005465446, 0.0053...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine...",<s> 0.213341 If 0.171282 Li...,0.172460,0.173025,0.176330


In [82]:
test_df4[['attention','tokens']] = test_df4.apply(average_attention,axis=1)
test_df4[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = test_df4.apply(roberta_key_attention_col,axis=1)
test_df4.to_csv('roberta_part5_type4_att_new.csv')
test_df4

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention,avg_cls_attention,key_phrase1_att,key_phrase2_att
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[0.17215282, 0.0038426423, 0.005423983, 0.005...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine...",<s> 0.213910 If 0.17259...,0.176148,0.172267,0.177741
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related,"[[0.16770327, 0.0035497255, 0.0051103025, 0.00...","[<s>, If, ĠBill, Ġis, Ġa, Ġcontent, Ġstrategis...",<s> 0.211913 If 0.16974...,0.170565,0.157983,0.172616
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related,"[[0.17198363, 0.0034583937, 0.0059485077, 0.00...","[<s>, If, ĠSteve, Ġis, Ġa, Ġconservation, Ġoff...",<s> 0.214726 If 0.169...,0.177441,0.176105,0.177265
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related,"[[0.16816372, 0.003677845, 0.0056913956, 0.004...","[<s>, If, ĠMatt, Ġis, Ġa, Ġsc, uba, Ġdiver, ,,...",<s> 0.213088 If 0.17212...,0.174726,0.331540,0.178796
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related,"[[0.17020833, 0.0036016905, 0.006539823, 0.004...","[<s>, If, ĠNoah, Ġis, Ġa, Ġpastry, Ġchef, ,, Ġ...",<s> 0.213853 If 0.169460 Noah...,0.175823,0.172018,0.209481
...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,"If Christina is a makeup artist, she'll grab h...",Christina has a makeup sponge.,E,possessive,her makeup sponge,rides a scooter,unrelated,"[[0.17304003, 0.0033935383, 0.0059805214, 0.00...","[<s>, If, ĠChristina, Ġis, Ġa, Ġmakeup, Ġartis...",<s> 0.214212 If 0.168300 Ch...,0.176559,0.176409,0.227104
293,"If Rahim is a clergyman, he'll bring his holy ...",Rahim has a holy book.,E,possessive,his holy book,gardens roses,unrelated,"[[0.17369306, 0.0037026766, 0.008242931, 0.0, ...","[<s>, If, ĠRah, im, Ġis, Ġa, Ġclergy, man, ,, ...",<s> 0.214490 If 0.167257 Ra...,0.175260,0.168897,0.188196
294,"If James is a police officer, he'll pick his g...",James has a gun.,E,possessive,his gun,drinks tea,unrelated,"[[0.17721617, 0.0036237594, 0.0058197654, 0.00...","[<s>, If, ĠJames, Ġis, Ġa, Ġpolice, Ġofficer, ...",<s> 0.217181 If 0.169872 James ...,0.180721,0.172149,0.187951
295,"If Maya is a piano teacher, she'll pack her me...",Maya has a metronome.,E,possessive,her metronome,knits hats,unrelated,"[[0.1704165, 0.003690166, 0.0066174604, 0.0052...","[<s>, If, ĠMaya, Ġis, Ġa, Ġpiano, Ġteacher, ,,...",<s> 0.211915 If 0.166234 Ma...,0.173900,0.334359,0.176047


In [83]:
test_df5a[['attention','tokens']] = test_df5a.apply(average_attention,axis=1)
test_df5a[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = test_df5a.apply(roberta_key_attention_col,axis=1)
test_df5a.to_csv('roberta_part5_type5a_att_new.csv')
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention,avg_cls_attention,key_phrase1_att,key_phrase2_att
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,attends a movie festival,watch a movie by Nolan again,related,"[[0.17383206, 0.0037895227, 0.0049331454, 0.00...","[<s>, If, ĠJessica, Ġattends, Ġa, Ġmovie, Ġfes...",<s> 0.213355 If 0.173393 Jess...,0.174271,0.169161,0.169157
1,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,finishes a mystery novel,read a book by Dan Brown again,related,"[[0.16946673, 0.0037922934, 0.0051234383, 0.00...","[<s>, If, ĠEmily, Ġfinishes, Ġa, Ġmystery, Ġno...",<s> 0.212047 If 0.172666 Emil...,0.171865,0.171517,0.166118
2,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,bakes a Swedish pastry,eat a traditional Swedish dish again,related,"[[0.16743962, 0.0044264817, 0.005428409, 0.009...","[<s>, If, ĠScarlett, Ġb, akes, Ġa, ĠSwedish, Ġ...",<s> 0.211344 If 0.16891...,0.174985,0.169769,0.173117
3,"If Chloe attends a pop concert, she'll never s...",Chloe has sung a pop song before.,E,again,attends a pop concert,sing a pop song again,related,"[[0.17697136, 0.003977977, 0.0071977293, 0.005...","[<s>, If, ĠChloe, Ġattends, Ġa, Ġpop, Ġconcert...",<s> 0.216108 If 0.177503 Chloe ...,0.179088,0.171998,0.173096
4,"If Charlotte attends a pop music festival, she...",Charlotte has listened to a pop song before.,E,again,attends a pop music festival,listen to a pop song again,related,"[[0.16895095, 0.0037841196, 0.006191456, 0.005...","[<s>, If, ĠCharlotte, Ġattends, Ġa, Ġpop, Ġmus...",<s> 0.212078 If 0.170065 Ch...,0.171770,0.168878,0.164917
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,again,breaks down,obtain a job position again,unrelated,"[[0.17003243, 0.0043898, 0.012263201, 0.0, 0.0...","[<s>, If, ĠDorothy, 's, Ġcar, Ġbreaks, Ġdown, ...",<s> 0.210872 If 0.173323 Do...,0.170262,0.165918,0.163791
796,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,again,finishes her work early,convey a confidential message again,unrelated,"[[0.17349827, 0.0040467894, 0.0056869434, 0.00...","[<s>, If, ĠSusan, Ġfinishes, Ġher, Ġwork, Ġear...",<s> 0.212866 If 0.173...,0.173833,0.162549,0.170058
797,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,finishes work early,manage to keep a secret again,unrelated,"[[0.16780135, 0.0039324234, 0.006933586, 0.005...","[<s>, If, ĠMargaret, Ġfinishes, Ġwork, Ġearly,...",<s> 0.210059 If 0.168734 Marg...,0.169308,0.165671,0.163658
798,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,misses the bus,cherish a moment in peace again,unrelated,"[[0.16978866, 0.0040895576, 0.0071632615, 0.00...","[<s>, If, ĠBetty, Ġmisses, Ġthe, Ġbus, ,, Ġshe...",<s> 0.211330 If 0.168926 Be...,0.170338,0.166145,0.164702


In [84]:
test_df5p[['attention','tokens']] = test_df5p.apply(average_attention,axis=1)
test_df5p[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = test_df5p.apply(roberta_key_attention_col,axis=1)
test_df5p.to_csv('roberta_part5_type5p_att_new.csv')
test_df5p

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention,avg_cls_attention,key_phrase1_att,key_phrase2_att
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,the university,related,"[[0.17674983, 0.004634804, 0.006193371, 0.0059...","[<s>, If, ĠJessica, Ġattends, Ġthe, Ġconferenc...",<s> 0.216783 If 0.173183 ...,0.176410,0.166303,0.161977
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,post exam celebratory dinner,related,"[[0.17173706, 0.003674858, 0.005905538, 0.0045...","[<s>, If, ĠEmily, Ġpasses, Ġher, Ġdriving, Ġte...",<s> 0.213571 If 0.16680...,0.169743,0.159538,0.171812
2,"If Scarlett finishes her chores early, her hus...",Scarlett has a husband.,E,possessive,her husband,lunch,related,"[[0.17417377, 0.004215937, 0.005900876, 0.0063...","[<s>, If, ĠScarlett, Ġfinishes, Ġher, Ġchores,...",<s> 0.216716 If 0.169768 Scar...,0.176568,0.164032,0.196249
3,"If Chloe finishes his project early, his colle...",Chloe has colleagues.,E,possessive,his colleagues,drinks,related,"[[0.17712025, 0.004594033, 0.008442057, 0.0061...","[<s>, If, ĠChloe, Ġfinishes, Ġhis, Ġproject, Ġ...",<s> 0.216956 If 0.168606 ...,0.175157,0.160894,0.192038
4,"If Charlotte passes her math test, her parents...",Charlotte has parents.,E,possessive,her parents,a calculator,related,"[[0.178489, 0.0046310555, 0.0077134296, 0.0060...","[<s>, If, ĠCharlotte, Ġpasses, Ġher, Ġmath, Ġt...",<s> 0.217947 If 0.171202 ...,0.180537,0.166934,0.184247
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated,"[[0.1751115, 0.004851301, 0.006780968, 0.00739...","[<s>, If, ĠDorothy, Ġbike, Ġbreaks, Ġdown, ,, ...",<s> 0.216147 If 0.173359 Doroth...,0.178066,0.163766,0.171227
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated,"[[0.17640345, 0.0040289722, 0.0055452753, 0.00...","[<s>, If, ĠSusan, Ġfinishes, Ġher, Ġassignment...",<s> 0.216935 If 0.171366 ...,0.176165,0.175567,0.171687
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated,"[[0.17341411, 0.004660236, 0.006708904, 0.0068...","[<s>, If, ĠMargaret, Ġfinishes, Ġwork, Ġearly,...",<s> 0.216072 If 0.168331 Marg...,0.175147,0.160993,0.164137
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated,"[[0.17396751, 0.0047013992, 0.0071969805, 0.00...","[<s>, If, ĠBetty, Ġmisses, Ġthe, Ġtrain, ,, Ġh...",<s> 0.215793 If 0.167220 Betty ...,0.175617,0.162597,0.168363


In [53]:
row = sample_test_df.iloc[0]
avg_attention = row['attention']
tokens = row['tokens']
attention_df = pd.DataFrame(avg_attention,index=tokens,columns=tokens)
attention_df

,<s>,If,ĠLiam,Ġis,Ġan,Ġaerospace,Ġengineer,",",Ġhe,'ll,...,</s>,</s>,L,iam,Ġhas,Ġa,Ġwind,Ġtunnel,.,</s>
<s>,0.172153,0.003843,0.005424,0.005183,0.004684,0.003756,0.003378,0.011287,0.008202,0.0,...,0.074501,0.041256,0.037764,0.0,0.041266,0.029409,0.013576,0.011266,0.232325,0.024938
If,0.147815,0.051543,0.033481,0.043885,0.022523,0.008421,0.010408,0.028887,0.045915,0.0,...,0.080238,0.018706,0.017308,0.0,0.020129,0.015748,0.004636,0.004329,0.182367,0.016445
ĠLiam,0.154061,0.019327,0.041766,0.027373,0.012028,0.008941,0.009374,0.011818,0.019411,0.0,...,0.087130,0.017844,0.044365,0.0,0.019281,0.015732,0.006165,0.004969,0.213932,0.015070
Ġis,0.134013,0.051372,0.043688,0.033911,0.043239,0.016635,0.031283,0.022191,0.045183,0.0,...,0.073768,0.014837,0.017043,0.0,0.022216,0.014671,0.004794,0.004735,0.166709,0.013814
Ġan,0.134626,0.030453,0.030124,0.044153,0.024281,0.044380,0.046160,0.020133,0.031471,0.0,...,0.068916,0.015060,0.018041,0.0,0.021675,0.018097,0.006679,0.006413,0.163196,0.013493
Ġaerospace,0.149009,0.005710,0.014800,0.010105,0.023412,0.030196,0.043861,0.009857,0.010773,0.0,...,0.081833,0.013211,0.013353,0.0,0.015999,0.015047,0.006683,0.006650,0.227333,0.010390
Ġengineer,0.134550,0.010764,0.020452,0.020382,0.023812,0.048089,0.035727,0.021202,0.018682,0.0,...,0.075907,0.013125,0.014663,0.0,0.017527,0.015586,0.007786,0.007887,0.199292,0.011219
",",0.130628,0.034575,0.014594,0.019409,0.012723,0.007726,0.018136,0.041903,0.061679,0.0,...,0.080672,0.030689,0.025561,0.0,0.031315,0.022357,0.008846,0.007612,0.170549,0.018439
Ġhe,0.252563,0.053014,0.050508,0.040878,0.023841,0.013296,0.021466,0.071764,0.164669,0.0,...,0.152717,0.038271,0.041039,0.0,0.048800,0.033408,0.011398,0.011178,0.332103,0.029392
'll,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [54]:
sample_test_df['cls_attention'] = sample_test_df.apply(roberta_key_attention_col,axis=1)
sample_test_df

,index,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention
0,0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[0.17215282, 0.0038426423, 0.005423983, 0.005...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine...","{'<s>': 0.2139100581407547, 'If': 0.1725986599..."
1,1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related,"[[0.16770327, 0.0035497255, 0.0051103025, 0.00...","[<s>, If, ĠBill, Ġis, Ġa, Ġcontent, Ġstrategis...","{'<s>': 0.21191348135471344, 'If': 0.169745519..."
2,2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related,"[[0.17198363, 0.0034583937, 0.0059485077, 0.00...","[<s>, If, ĠSteve, Ġis, Ġa, Ġconservation, Ġoff...","{'<s>': 0.21472637355327606, 'If': 0.169642046..."
3,3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related,"[[0.16816372, 0.003677845, 0.0056913956, 0.004...","[<s>, If, ĠMatt, Ġis, Ġa, Ġsc, uba, Ġdiver, ,,...","{'<s>': 0.21308791637420654, 'If': 0.172129109..."
4,4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related,"[[0.17020833, 0.0036016905, 0.006539823, 0.004...","[<s>, If, ĠNoah, Ġis, Ġa, Ġpastry, Ġchef, ,, Ġ...","{'<s>': 0.2138526886701584, 'If': 0.1694602221..."
5,5,"If Donald is a crime scene investigator, he'll...",Donald has evidence bags.,E,possessive,his evidence bags,secure crime scene samples,related,"[[0.17583454, 0.003670351, 0.005651432, 0.0048...","[<s>, If, ĠDonald, Ġis, Ġa, Ġcrime, Ġscene, Ġi...","{'<s>': 0.21514952182769775, 'If': 0.171464189..."
6,6,"If Edward is a playwright, he'll use his pen t...",Edward has a pen.,E,possessive,his pen,write scripts for the stage,related,"[[0.17331609, 0.00359642, 0.00496802, 0.004719...","[<s>, If, ĠEdward, Ġis, Ġa, Ġplay, wright, ,, ...","{'<s>': 0.2154555320739746, 'If': 0.1708635240..."
7,7,"If William is a notary, he'll use his notary s...",William has a notary seal.,E,possessive,his notary seal,certify documents,related,"[[0.1739344, 0.004389406, 0.0056128902, 0.0050...","[<s>, If, ĠWilliam, Ġis, Ġa, Ġnot, ary, ,, Ġhe...","{'<s>': 0.21510501205921173, 'If': 0.172613203..."


In [41]:
test_df4[['attention','tokens']] = test_df4.apply(average_attention,axis=1)
test_df4['cls_attention'] = test_df4.apply(cls_token_attention, axis=1)
test_df4

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[0.17215282, 0.0038426423, 0.005423983, 0.005...","[<s>, If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengine...","{'<s>': 0.1721528172492981, 'If': 0.0038426422..."
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related,"[[0.16770327, 0.0035497255, 0.0051103025, 0.00...","[<s>, If, ĠBill, Ġis, Ġa, Ġcontent, Ġstrategis...","{'<s>': 0.1677032709121704, 'If': 0.0035497255..."
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related,"[[0.17198363, 0.0034583937, 0.0059485077, 0.00...","[<s>, If, ĠSteve, Ġis, Ġa, Ġconservation, Ġoff...","{'<s>': 0.17198362946510315, 'If': 0.003458393..."
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related,"[[0.16816372, 0.003677845, 0.0056913956, 0.004...","[<s>, If, ĠMatt, Ġis, Ġa, Ġsc, uba, Ġdiver, ,,...","{'<s>': 0.1681637167930603, 'If': 0.0036778450..."
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related,"[[0.17020833, 0.0036016905, 0.006539823, 0.004...","[<s>, If, ĠNoah, Ġis, Ġa, Ġpastry, Ġchef, ,, Ġ...","{'<s>': 0.17020833492279053, 'If': 0.003601690..."
...,...,...,...,...,...,...,...,...,...,...
292,"If Christina is a makeup artist, she'll grab h...",Christina has a makeup sponge.,E,possessive,her makeup sponge,rides a scooter,unrelated,"[[0.17304003, 0.0033935383, 0.0059805214, 0.00...","[<s>, If, ĠChristina, Ġis, Ġa, Ġmakeup, Ġartis...","{'<s>': 0.17304003238677979, 'If': 0.003393538..."
293,"If Rahim is a clergyman, he'll bring his holy ...",Rahim has a holy book.,E,possessive,his holy book,gardens roses,unrelated,"[[0.17369306, 0.0037026766, 0.008242931, 0.0, ...","[<s>, If, ĠRah, im, Ġis, Ġa, Ġclergy, man, ,, ...","{'<s>': 0.17369306087493896, 'If': 0.003702676..."
294,"If James is a police officer, he'll pick his g...",James has a gun.,E,possessive,his gun,drinks tea,unrelated,"[[0.17721617, 0.0036237594, 0.0058197654, 0.00...","[<s>, If, ĠJames, Ġis, Ġa, Ġpolice, Ġofficer, ...","{'<s>': 0.17721617221832275, 'If': 0.003623759..."
295,"If Maya is a piano teacher, she'll pack her me...",Maya has a metronome.,E,possessive,her metronome,knits hats,unrelated,"[[0.1704165, 0.003690166, 0.0066174604, 0.0052...","[<s>, If, ĠMaya, Ġis, Ġa, Ġpiano, Ġteacher, ,,...","{'<s>': 0.17041650414466858, 'If': 0.003690165..."


In [42]:
test_df4.to_csv('roberta_part5_type4_cls.csv')

In [43]:
test_df5a[['attention','tokens']] = test_df5a.apply(average_attention,axis=1)
test_df5a['cls_attention'] = test_df5a.apply(cls_token_attention, axis=1)
test_df5a.to_csv('roberta_part5_type5a_cls.csv')
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,attends a movie festival,watch a movie by Nolan again,related,"[[0.17383206, 0.0037895227, 0.0049331454, 0.00...","[<s>, If, ĠJessica, Ġattends, Ġa, Ġmovie, Ġfes...","{'<s>': 0.17383205890655518, 'If': 0.003789522..."
1,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,finishes a mystery novel,read a book by Dan Brown again,related,"[[0.16946673, 0.0037922934, 0.0051234383, 0.00...","[<s>, If, ĠEmily, Ġfinishes, Ġa, Ġmystery, Ġno...","{'<s>': 0.16946673393249512, 'If': 0.003792293..."
2,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,bakes a Swedish pastry,eat a traditional Swedish dish again,related,"[[0.16743962, 0.0044264817, 0.005428409, 0.009...","[<s>, If, ĠScarlett, Ġb, akes, Ġa, ĠSwedish, Ġ...","{'<s>': 0.16743962466716766, 'If': 0.004426481..."
3,"If Chloe attends a pop concert, she'll never s...",Chloe has sung a pop song before.,E,again,attends a pop concert,sing a pop song again,related,"[[0.17697136, 0.003977977, 0.0071977293, 0.005...","[<s>, If, ĠChloe, Ġattends, Ġa, Ġpop, Ġconcert...","{'<s>': 0.17697136104106903, 'If': 0.003977977..."
4,"If Charlotte attends a pop music festival, she...",Charlotte has listened to a pop song before.,E,again,attends a pop music festival,listen to a pop song again,related,"[[0.16895095, 0.0037841196, 0.006191456, 0.005...","[<s>, If, ĠCharlotte, Ġattends, Ġa, Ġpop, Ġmus...","{'<s>': 0.16895094513893127, 'If': 0.003784119..."
...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,again,breaks down,obtain a job position again,unrelated,"[[0.17003243, 0.0043898, 0.012263201, 0.0, 0.0...","[<s>, If, ĠDorothy, 's, Ġcar, Ġbreaks, Ġdown, ...","{'<s>': 0.17003242671489716, 'If': 0.004389800..."
796,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,again,finishes her work early,convey a confidential message again,unrelated,"[[0.17349827, 0.0040467894, 0.0056869434, 0.00...","[<s>, If, ĠSusan, Ġfinishes, Ġher, Ġwork, Ġear...","{'<s>': 0.173498272895813, 'If': 0.00404678937..."
797,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,finishes work early,manage to keep a secret again,unrelated,"[[0.16780135, 0.0039324234, 0.006933586, 0.005...","[<s>, If, ĠMargaret, Ġfinishes, Ġwork, Ġearly,...","{'<s>': 0.16780135035514832, 'If': 0.003932423..."
798,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,misses the bus,cherish a moment in peace again,unrelated,"[[0.16978866, 0.0040895576, 0.0071632615, 0.00...","[<s>, If, ĠBetty, Ġmisses, Ġthe, Ġbus, ,, Ġshe...","{'<s>': 0.169788658618927, 'If': 0.00408955756..."


In [44]:
test_df5p[['attention','tokens']] = test_df5p.apply(average_attention,axis=1)
test_df5p['cls_attention'] = test_df5p.apply(cls_token_attention, axis=1)
test_df5p.to_csv('roberta_part5_type5p_cls.csv')
test_df5p

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,the university,related,"[[0.17674983, 0.004634804, 0.006193371, 0.0059...","[<s>, If, ĠJessica, Ġattends, Ġthe, Ġconferenc...","{'<s>': 0.1767498254776001, 'If': 0.0046348040..."
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,post exam celebratory dinner,related,"[[0.17173706, 0.003674858, 0.005905538, 0.0045...","[<s>, If, ĠEmily, Ġpasses, Ġher, Ġdriving, Ġte...","{'<s>': 0.17173705995082855, 'If': 0.003674858..."
2,"If Scarlett finishes her chores early, her hus...",Scarlett has a husband.,E,possessive,her husband,lunch,related,"[[0.17417377, 0.004215937, 0.005900876, 0.0063...","[<s>, If, ĠScarlett, Ġfinishes, Ġher, Ġchores,...","{'<s>': 0.1741737723350525, 'If': 0.0042159371..."
3,"If Chloe finishes his project early, his colle...",Chloe has colleagues.,E,possessive,his colleagues,drinks,related,"[[0.17712025, 0.004594033, 0.008442057, 0.0061...","[<s>, If, ĠChloe, Ġfinishes, Ġhis, Ġproject, Ġ...","{'<s>': 0.17712025344371796, 'If': 0.004594033..."
4,"If Charlotte passes her math test, her parents...",Charlotte has parents.,E,possessive,her parents,a calculator,related,"[[0.178489, 0.0046310555, 0.0077134296, 0.0060...","[<s>, If, ĠCharlotte, Ġpasses, Ġher, Ġmath, Ġt...","{'<s>': 0.17848899960517883, 'If': 0.004631055..."
...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated,"[[0.1751115, 0.004851301, 0.006780968, 0.00739...","[<s>, If, ĠDorothy, Ġbike, Ġbreaks, Ġdown, ,, ...","{'<s>': 0.17511150240898132, 'If': 0.004851301..."
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated,"[[0.17640345, 0.0040289722, 0.0055452753, 0.00...","[<s>, If, ĠSusan, Ġfinishes, Ġher, Ġassignment...","{'<s>': 0.1764034479856491, 'If': 0.0040289722..."
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated,"[[0.17341411, 0.004660236, 0.006708904, 0.0068...","[<s>, If, ĠMargaret, Ġfinishes, Ġwork, Ġearly,...","{'<s>': 0.17341411113739014, 'If': 0.004660236..."
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated,"[[0.17396751, 0.0047013992, 0.0071969805, 0.00...","[<s>, If, ĠBetty, Ġmisses, Ġthe, Ġtrain, ,, Ġh...","{'<s>': 0.17396751046180725, 'If': 0.004701399..."
